In [ ]:
# 1. IMPOR LIBRARY
# Impor semua fungsi yang kita butuhkan dari fastai
from fastai.vision.all import *

print("Library fastai berhasil diimpor.")

# 2. TENTUKAN PATH DATA ANDA
# Ganti 'train_labeled/' dengan nama folder Anda yang berisi
# 15 folder makanan yang sudah dilabeli.
path = Path('train_labeled/')

# Cek apakah path valid
if not path.exists():
    print(f"Error: Path '{path}' tidak ditemukan. Harap periksa nama folder Anda.")
else:
    print(f"Path data ditemukan: {path}")

    # 3. BUAT DATALOADERS (Buku Contekan + Kuis)
    # Ini akan memuat gambar, memberi label dari nama folder,
    # dan membaginya 80% latih, 20% validasi (kuis).
    dls = ImageDataLoaders.from_folder(
        path,
        valid_pct=0.2,       # Sisihkan 20% data untuk kuis (validasi)
        seed=42,             # 'seed=42' agar pembagiannya selalu sama setiap kali dijalankan
        item_tfms=RandomResizedCrop(224, min_scale=0.75), # Augmentasi: crop acak
        batch_tfms=aug_transforms()  # Augmentasi: putar, balik, ubah pencahayaan
    )

    # Opsional: Tampilkan beberapa contoh gambar untuk cek
    print("Menampilkan beberapa contoh gambar dari data latih...")
    dls.show_batch(max_n=9)

    # 4. BUAT MODEL (Rekrut Intern)
    # Kita gunakan arsitektur 'efficientnet_b0' yang sudah dilatih di ImageNet
    # Kita ingin melacak 'accuracy' (akurasi)
    learn = vision_learner(
        dls,
        'timm.efficientnet_b0',
        metrics=accuracy
    )

    print("Model (learner) berhasil dibuat.")

    # 5. LATIH MODEL (Proses Belajar)
    # 'fine_tune(5)' adalah cara cerdas untuk melatih:
    # 1 epoch melatih "kepala" baru model
    # 5 epoch melatih seluruh model dengan hati-hati
    print("Memulai pelatihan model baseline (5 epoch)...")
    learn.fine_tune(5)
    print("Pelatihan selesai.")

    # 6. AUDIT KESALAHAN (Bagian Paling PentING!)
    # Buat objek 'Interpretation' untuk menganalisis hasil kuis (validasi)
    print("Menganalisis hasil model...")
    interp = ClassificationInterpretation.from_learner(learn)

    # 7. TAMPILKAN CONFUSION MATRIX
    # Ini menunjukkan kelas mana yang sering tertukar
    # (misal: Rendang sering salah tebak jadi Kalio)
    print("Menampilkan Confusion Matrix...")
    interp.plot_confusion_matrix(figsize=(10,10))

    # 8. TAMPILKAN KESALAHAN TERPARAH
    # Ini adalah alat audit utama Anda.
    # 'k=16' berarti tampilkan 16 kesalahan terparah.
    print("Menampilkan 16 Kesalahan Teratas (plot_top_losses)...")
    interp.plot_top_losses(k=16, nrows=4)